In [1]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import numpy as np

images = [basic_ops.load_image(image_path='traffic_sign_images/vfs/vfs_04.jpg')]
images = [geo_ops.resize_image(image=image, target_height=image.shape[0]//8, target_width=image.shape[1]//8) for image in images]

Image loaded from traffic_sign_images/vfs/vfs_04.jpg


In [2]:
for i, image in enumerate(images):
    image = filters.median_filter(image=image, dim=3)
    #image = stat_ops.gamma_equalization(image=image, gamma=0.25)
    images[i] = image

In [3]:
import importlib
import colors
import bounding_box
import color_detection
importlib.reload(color_detection)
importlib.reload(colors)
importlib.reload(bounding_box)

color_functions = [colors.is_strong_red, colors.is_strong_yellow, colors.is_strong_blue]
color_bounding_boxes = []
for i, image in enumerate(images):
    for color_function in color_functions:
        mask = colors.get_mask(image=image, color_function=color_function)
        blobs = color_detection.get_blobs(mask=mask)

        box_color = colors.get_color_from_function(color_function)
        height, width = image.shape[:2]
        min_box_area = int((height*0.055)**2)
        max_box_area = height*width//2

        bounding_boxes = bounding_box.create_bounding_boxes(blobs=blobs, min_box_area=min_box_area, max_box_area=max_box_area, box_color=box_color, image_index=i)
        color_bounding_boxes.extend(bounding_boxes)

In [4]:
import copy

bbox_images = copy.deepcopy(images)
for bounding_box_obj in color_bounding_boxes:
    print(bounding_box_obj)
    bbox_image = bounding_box.draw_bounding_box(bounding_box=bounding_box_obj, image=bbox_images[bounding_box_obj.image_index])
    bbox_images[bounding_box_obj.image_index] = bbox_image

%store color_bounding_boxes

BoundingBox(center_y=27, center_x=24, box_corners=[1, 0, 53, 49], height=53, width=50, area=2650, box_color=[0, 255, 255], image_index=0)
BoundingBox(center_y=185, center_x=231, box_corners=[166, 214, 204, 249], height=39, width=36, area=1404, box_color=[0, 255, 255], image_index=0)
Stored 'color_bounding_boxes' (list)


In [5]:
for image in bbox_images:
    basic_ops.show_image(image=image, title='bbox image')

Image displayed
